In [1]:
from __future__ import print_function, division
import os
import torch
import json
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import pdb

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

from io import BytesIO
import urllib
from PIL import Image


In [2]:
path = '/home/edwin/Datasets/competitions/imaterialist-challenge-furniture-2018/'

In [3]:
num_epochs = 2

In [4]:
def get_image_id(x): return x['image_id']
def get_url(x): return x['url'][0]
def get_label_id(x): return x['label_id']

In [5]:
train_path = f'{path}train.json'

In [6]:
with open(train_path) as d1:
    train_json = json.load(d1)

In [ ]:
labels = list(map(get_label_id, train_json['annotations']))

In [ ]:
# of unique labels
classes = set(labels)
num_classes = len(classes)

In [ ]:
images_sample = train_json['images']
annotations_sample = train_json['annotations']

In [ ]:
ids = list(map(get_image_id, images_sample))
images = list(map(get_url, images_sample))
labels = list(map(get_label_id, annotations_sample))

In [ ]:
data = {'image_id': ids, 'images': images, 'labels': labels}

In [ ]:
data

In [7]:
df = pd.DataFrame(data=data)

NameError: name 'data' is not defined

In [ ]:
df.to_pickle('df_full.pkl')

In [ ]:
df.to_pickle('imaterialist_full.pkl')

In [ ]:
df = pd.read_pickle('imaterialist_full.pkl')

In [ ]:
df.images.iloc[0]

In [ ]:
! wget -O t.jpg "{df.images.iloc[0]}"

In [ ]:
! ls 

In [4]:
df = pd.read_pickle('df_full.pkl')

In [9]:
len(df)

194828

In [10]:
np.random.randn

<function RandomState.randn>

In [12]:
# creating a random dataset from pandas df
msk = np.random.rand(len(df)) < 0.05

In [13]:
len(msk)

194828

In [14]:
sample_train = df[msk]

In [15]:
test_msk = np.random.rand(len(df)) < 0.005
sample_test = df[test_msk]

In [16]:
len(sample_test)

981

In [17]:
len(sample_train)

9721

In [18]:
train = df[msk]

In [19]:
len(train)

9721

In [20]:
train

,image_id,images,labels
8,9,http://dimg.cnlist.org/image/upload/68/3f/c1/9...,5
51,52,https://img13.360buyimg.com/imgzone/jfs/t3910/...,5
60,61,https://img.alicdn.com/imgextra/TB2zB0OamJjpuF...,5
72,73,https://img14.360buyimg.com/imgzone/jfs/t4636/...,5
130,131,http://img11.360buyimg.com/popWaterMark/jfs/t1...,5
158,159,http://www.62a.net/tbimg/img/imgextra/i1/17534...,5
164,165,https://img.alicdn.com/imgextra/TB2t48wdjgy_uJ...,5
194,195,http://image.it168.com/n/640x480/6/6174/617459...,5
259,260,https://img.alicdn.com/imgextra/TB2GZh.mb8kpuF...,5
273,274,http://m.360buyimg.com/n12/jfs/t2584/167/33818...,5


In [21]:
df1 = pd.read_pickle("df1.pkl")

In [5]:
IMAGES_TRAIN_PATH = "/home/edwin/Datasets/imaterialist-challenge-furniture-2018-sample/train"

In [6]:
class IMaterialDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        image_id = self.df.iloc[idx].image_id
        try:
            file = f"{IMAGES_TRAIN_PATH}/{image_id}.jpg"
            img = Image.open(file)
            img = np.asarray(img)
        except:
            img = np.zeros([800, 800, 3])
            
        label = self.df['labels'].iloc[idx]
        sample = {'image': img, 'label': label}
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample

In [7]:
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (74, 74), mode='constant')

        return {'image': img, 'label': label}
    
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        
        return {'image': torch.from_numpy(image).float(),
                'label': torch.from_numpy(np.array([label]))}
    

In [22]:
composed = transforms.Compose([Rescale(50), ToTensor()])

In [23]:
ds = IMaterialDataset(df=sample_train, transform=composed)

In [24]:
ds_test = IMaterialDataset(df=sample_test, transform=composed)

In [25]:
len(ds), len(ds_test)

(9721, 981)

In [26]:
dataloader = DataLoader(ds, batch_size=4, shuffle=True, num_workers=4)

In [27]:
testloader = DataLoader(ds_test, batch_size=4, shuffle=True, num_workers=4)

In [ ]:
len(testloader)

In [28]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [29]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 10, 5)
        self.fc1 = nn.Linear(2250 , 1000)
        self.fc2 = nn.Linear(1000, 500)
        self.fc3 = nn.Linear(500, 128)
        
    def forward(self, x):
        # original input torch.Size([4, 3, 75, 75])
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [30]:
cnn = CNNModel()

In [31]:
cnn.cuda()

CNNModel(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (conv2): Conv2d(6, 10, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=2250, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=128, bias=True)
)

In [32]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.001, momentum=0.9)

In [33]:
data['label']

NameError: name 'data' is not defined

In [ ]:
torch.cuda.is_available()

In [34]:
iter = 0
for i, data in enumerate(dataloader, 0):
    images = Variable(data['image'].cuda())
    labels = Variable(data['label'].cuda())

    optimizer.zero_grad()

    outputs = cnn(images)
    loss = criterion(outputs, labels.view(-1))
    loss.backward()
    optimizer.step()
    iter += 1
    if iter % 500 == 0:
        correct = 0
        total = 0
        for data_train in testloader:
            images = Variable(data_train['image'].cuda())
            labels = data_train['label']

            test_outputs = cnn(images)

            _, predicted = torch.max(test_outputs.data, 1)

            total += labels.size(0)
            correct += (predicted.cpu() == labels.view(-1)).sum()
        accuracy = 100 * correct / total
        print('Correct:{}, Total:{}'.format(correct, total))
        print('Iteraton:{}, Loss:{}, Accuracy: {}'.format(iter, loss.data[0], accuracy))    
    
        

RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1522182087074/work/torch/lib/THCUNN/generic/ClassNLLCriterion.cu:113

In [ ]:
% debug

> /home/edwin/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py(1052)nll_loss()
   1050         weight = Variable(weight)
   1051     if dim == 2:
-> 1052         return torch._C._nn.nll_loss(input, target, weight, size_average, ignore_index, reduce)
   1053     elif dim == 4:
   1054         return torch._C._nn.nll_loss2d(input, target, weight, size_average, ignore_index, reduce)

ipdb> weight
ipdb> weight.size
*** AttributeError: 'NoneType' object has no attribute 'size'
ipdb> weight.shape
*** AttributeError: 'NoneType' object has no attribute 'shape'


In [33]:
Variable(data['image'].cuda())

RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1522182087074/work/torch/lib/THC/generic/THCTensorCopy.c:20

In [ ]:
Image.open('/home/edwin/Datasets/imaterialist-challenge-furniture-2018-sample/train/108880.jpg')

In [ ]:
Variable(data['image'].cuda())

In [ ]:
print(torch.__version__)

In [ ]:
_, predicted = torch.max(outputs.data, 1)

In [ ]:
predicted

In [ ]:
sample['label']

In [ ]:
(predicted == sample['label']).sum()

In [ ]:
labels.view(-1)

In [ ]:
labels.view(-1)

In [ ]:
dataiter = iter(dataloader)

In [ ]:
sample = dataiter.next()

In [ ]:
images = Variable(sample['image'])

In [ ]:
images.shape

In [ ]:
outputs = cnn(images)

In [ ]:
labels = Variable(sample['label'].view(4))